In [36]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [37]:
batch_size = 128
test_size = 256

In [38]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

In [39]:
def model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden):
    l1a = tf.nn.relu(tf.nn.conv2d(X, w,                       # l1a shape=(?, 28, 28, 32)
                        strides=[1, 1, 1, 1], padding='SAME'))
    l1 = tf.nn.max_pool(l1a, ksize=[1, 2, 2, 1],              # l1 shape=(?, 14, 14, 32)
                        strides=[1, 2, 2, 1], padding='SAME')
    l1 = tf.nn.dropout(l1, p_keep_conv)

    l2a = tf.nn.relu(tf.nn.conv2d(l1, w2,                     # l2a shape=(?, 14, 14, 64)
                        strides=[1, 1, 1, 1], padding='SAME'))
    l2 = tf.nn.max_pool(l2a, ksize=[1, 2, 2, 1],              # l2 shape=(?, 7, 7, 64)
                        strides=[1, 2, 2, 1], padding='SAME')
    l2 = tf.nn.dropout(l2, p_keep_conv)

    l3a = tf.nn.relu(tf.nn.conv2d(l2, w3,                     # l3a shape=(?, 7, 7, 128)
                        strides=[1, 1, 1, 1], padding='SAME'))
    l3 = tf.nn.max_pool(l3a, ksize=[1, 2, 2, 1],              # l3 shape=(?, 4, 4, 128)
                        strides=[1, 2, 2, 1], padding='SAME')
    l3 = tf.reshape(l3, [-1, w4.get_shape().as_list()[0]])    # reshape to (?, 2048)
    l3 = tf.nn.dropout(l3, p_keep_conv)

    l4 = tf.nn.relu(tf.matmul(l3, w4))
    l4 = tf.nn.dropout(l4, p_keep_hidden)

    pyx = tf.matmul(l4, w_o)
    return pyx

In [40]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels
trX = trX.reshape(-1, 28, 28, 1)  # 28x28x1 input img
teX = teX.reshape(-1, 28, 28, 1)  # 28x28x1 input img

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [41]:
X = tf.placeholder("float", [None, 28, 28, 1])
Y = tf.placeholder("float", [None, 10])


w = init_weights([3, 3, 1, 32])       # 3x3x1 conv, 32 outputs
w2 = init_weights([3, 3, 32, 64])     # 3x3x32 conv, 64 outputs
w3 = init_weights([3, 3, 64, 128])    # 3x3x32 conv, 128 outputs
w4 = init_weights([128 * 4 * 4, 625]) # FC 128 * 4 * 4 inputs, 625 outputs
w_o = init_weights([625, 10])         # FC 625 inputs, 10 outputs (labels)

p_keep_conv = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")
py_x = model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y))
train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict_op = tf.argmax(py_x, 1)

In [42]:
# Launch the graph in a session
with tf.Session() as sess:
    # you need to initialize all variables
    tf.global_variables_initializer().run()

    for i in range(100):
        training_batch = zip(range(0, len(trX), batch_size),
                             range(batch_size, len(trX)+1, batch_size))
        for start, end in training_batch:
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end],
                                          p_keep_conv: 0.8, p_keep_hidden: 0.5})

        test_indices = np.arange(len(teX)) # Get A Test Batch
        np.random.shuffle(test_indices)
        test_indices = test_indices[0:test_size]

        print(i, np.mean(np.argmax(teY[test_indices], axis=1) ==
                         sess.run(predict_op, feed_dict={X: teX[test_indices],
                                                         Y: teY[test_indices],
                                                         p_keep_conv: 1.0,
                                                         p_keep_hidden: 1.0})))

0 0.97265625
1 0.9921875
2 0.99609375
3 0.9921875
4 0.9921875


KeyboardInterrupt: 

# my model

In [77]:
def mini_model(X, w, wb , w4, w_o, p_keep_conv, p_keep_hidden):
    l1a = tf.nn.relu(tf.nn.conv2d(X, w,                       # l1a shape=(?, 28, 28, 32)
                        strides=[1, 1, 1, 1], padding='SAME'))
    l1 = tf.nn.max_pool(l1a, ksize=[1, 2, 2, 1],              # l1 shape=(?, 14, 14, 32)
                        strides=[1, 2, 2, 1], padding='SAME')
    lba = tf.nn.relu(tf.nn.conv2d(l1 , wb ,                                   
                                  strides = [1,1,1,1] ,
                                 padding = 'SAME'))
    lb = tf.nn.max_pool(lba , 
                        ksize = [1,2,2,1] , 
                        strides = [1,2,2,1] , 
                        padding = 'SAME'
                       )
    l3 = tf.reshape(lb, [-1, w4.get_shape().as_list()[0]])    # reshape to (?, 2048)
    l3 = tf.nn.dropout(l3, p_keep_conv)

    l4 = tf.nn.relu(tf.matmul(l3, w4))
    l4 = tf.nn.dropout(l4, p_keep_hidden)

    pyx = tf.matmul(l4, w_o)
    return pyx

In [78]:
X = tf.placeholder("float", [None, 28, 28, 1])
Y = tf.placeholder("float", [None, 10])

n_channel1 = 4
n_channel2 = 8
n_flattened = 25

w = init_weights([3, 3, 1, n_channel1])       # 3x3x1 conv, 32 outputs
wb = init_weights([3 , 3, n_channel1 , n_channel2])
w4 = init_weights([n_channel2 * 7 * 7, n_flattened]) # FC 128 * 4 * 4 inputs, 625 outputs
w_o = init_weights([n_flattened, 10])         # FC 625 inputs, 10 outputs (labels)

p_keep_conv = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")
py_x = mini_model(X, w, wb, w4, w_o, p_keep_conv, p_keep_hidden)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y))
train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict_op = tf.argmax(py_x, 1)

In [79]:
sess = tf.Session()

sess.run(tf.global_variables_initializer())

In [80]:
sess.run(py_x, feed_dict={X: trX[0:100], p_keep_conv: 0.8, p_keep_hidden: 0.5})

array([[ 3.52338930e-06,  1.56449471e-06, -7.23587164e-07,
        -3.25246856e-06, -7.77462560e-07, -3.69167583e-06,
         1.62120341e-06, -7.89485512e-07, -5.65023583e-09,
         1.12688613e-05],
       [ 2.94922893e-06,  3.04203195e-06, -7.55755491e-06,
         3.14922522e-06,  4.95387985e-06, -1.07834931e-05,
         4.31778153e-06,  1.50059668e-06, -9.70670271e-07,
         2.23917937e-06],
       [-4.45334081e-07, -1.23580537e-06,  4.25212511e-06,
        -1.92825405e-06, -1.93127357e-06,  2.35269954e-06,
         3.22409051e-07,  1.78801338e-06, -2.33111723e-06,
         1.73240892e-06],
       [ 4.14127453e-06,  4.46110835e-06, -2.43306431e-07,
         1.46791490e-06, -1.72316857e-06, -1.79009544e-06,
         3.19135006e-06, -3.38627217e-07, -4.46023080e-07,
         3.03945581e-06],
       [ 1.04589390e-06, -3.36660037e-07,  8.30808517e-07,
        -4.01613397e-06, -1.53796191e-06,  1.42252281e-06,
         3.39555072e-06,  1.64583309e-06, -2.68904432e-06,
         2.

In [81]:
sess.run(py_x, feed_dict={X: trX[0:10],p_keep_conv: 0.8, p_keep_hidden: 0.5}).shape

(10, 10)

In [63]:
(np.argmax(trY[0:10], axis=1))

array([7, 3, 4, 6, 1, 8, 1, 0, 9, 8], dtype=int64)

In [ ]:
tf.global_variables_initializer().run()

In [82]:
for i in range(20):
    training_batch = zip(range(0, len(trX), batch_size),
                         range(batch_size, len(trX)+1, batch_size))
    for start, end in training_batch:
        sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end],
                                      p_keep_conv: 0.8, p_keep_hidden: 0.5})

    test_indices = np.arange(len(teX)) # Get A Test Batch
    np.random.shuffle(test_indices)
    test_indices = test_indices[0:test_size]

    print(i, np.mean(np.argmax(teY[test_indices], axis=1) ==
                     sess.run(predict_op, feed_dict={X: teX[test_indices],
                                                     Y: teY[test_indices],
                                                     p_keep_conv: 1.0,
                                                     p_keep_hidden: 1.0})))

0 0.71484375
1 0.8515625
2 0.88671875
3 0.90625
4 0.92578125
5 0.94921875
6 0.95703125
7 0.96875
8 0.98046875
9 0.97265625
10 0.97265625
11 0.96484375
12 0.96875
13 0.96484375
14 0.98046875
15 0.9765625
16 0.96875
17 0.98828125
18 0.953125
19 0.97265625
20 0.9765625
21 0.98828125
22 0.97265625
23 0.97265625
24 0.94921875
25 0.98046875
26 0.9765625
27 0.9765625
28 0.984375
29 0.96875
30 0.984375
31 0.984375


KeyboardInterrupt: 

# 解説

## 次元

defaultの"NHWC"の場合、
[batch, height, width, channels]

## conv2d

[filter_height, filter_width, in_channels, out_channels]

の４次元テンソルを与える。

## max_pool

https://qiita.com/tadOne/items/b484ce9f973a9f80036e

http://7rpn.hatenablog.com/entry/2015/11/29/131601

In [9]:
?tf.nn.max_pool

  
  |argument|description|
  |:----|:----|
  |value| A 4-D `Tensor` of the format specified by `data_format`.|
  |ksize| A 1-D int Tensor of 4 elements.  The size of the window for each dimension of the input tensor.|
  |strides| A 1-D int Tensor of 4 elements.  The stride of the sliding window for each dimension of the input tensor.|
  |padding| A string, either `'VALID'` or `'SAME'`. The padding algorithm.|
|  data_format| A string. 'NHWC', 'NCHW' and 'NCHW_VECT_C' are supported.|
 | name| Optional name for the operation.|

- ksizeが(1,10,20,1)のとき、縦方向(?)に10個のノードのmaxを取って返す
- stridesが(1,2,3,1)のとき、縦方向（？）には２＝ひとつ置きに１０個(上記のksizeの場合)のノードのmaxを返すので、その方向での結果の次元はざっくり半分になる。

参考：
strides = [1, 2, 2, 1] applies the filter to every other image patch in each dimension
https://www.tensorflow.org/api_guides/python/nn#Pooling



## tf.reshape

If one component of shape is the special value -1, the size of that dimension is computed so that the total size remains constant. In particular, a shape of [-1] flattens into 1-D. At most one component of shape can be -1.

# sandbox